In [ ]:
import numpy as np
%tensorflow_version 1.x
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

In [ ]:
import pandas as pd
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "utf-8",
       names = ["Sentence", "Intent"])
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)

In [ ]:
intent, unique_intent, sentences = load_dataset("/content/bank.csv")

In [ ]:
print(intent[:5])

0    2in1_acoount_info
1    2in1_acoount_info
2    2in1_acoount_info
3    2in1_acoount_info
4    2in1_acoount_info
Name: Intent, dtype: object


In [ ]:
nltk.download("punkt")
def cleaning(sentences):
  words = [] 
  for s in sentences:
    w = word_tokenize(s)
    words.append([i for i in w])     
  return words  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
print(len(sentences))

1567
[['நான்', '2in1', 'கணக்கில்', 'சேமிப்பது', 'எப்படி', '?'], ['2in1', 'கணக்கில்', 'நான்', 'சேமிப்பது', 'எப்படி', '?']]
1567


In [ ]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [ ]:
def max_length(words):
  return(len(max(words, key = len)))

In [ ]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 1141 and Maximum length = 22


In [ ]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [ ]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [ ]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [ ]:
padded_doc = padding_doc(encoded_doc, max_length)

In [ ]:
padded_doc[:5]

array([[  3,  45,  62, 348,  11,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 45,  62,   3, 348,  11,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 11,   3,  45,  62, 348,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  3,  11,  45,  62, 348,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 45,  62,   3, 587, 287,   2,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [ ]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (1567, 22)


In [ ]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')

In [ ]:
output_tokenizer.word_index

{'2in1_acoount_info': 7,
 '2in1_atm_card': 1,
 '2in1_interest_receiving': 46,
 '2in1_min_balance': 17,
 '2in1_pass_book': 47,
 'account_currency': 20,
 'bank_statement_online': 35,
 'cancel_card': 56,
 'card_automatic_renewal': 34,
 'card_foreign_use': 23,
 'card_machine_repair': 8,
 'card_usage': 43,
 'change_details': 14,
 'cvv_use': 13,
 'debit_card_requirement': 2,
 'fcaispe_required_docs': 50,
 'foreign_account_lkr_withdrawal': 3,
 'foreign_currency_withdrawal': 45,
 'foreign_currency_withdrawal_currency': 22,
 'foreign_deposit_loan': 9,
 'get_lc_form': 51,
 'get_monthly_report': 30,
 'housing_loan_documents': 5,
 'housing_loan_purpose': 54,
 'interest_credit_info': 44,
 'internet_bank_loan_amount': 36,
 'joint_account_details': 25,
 'life_insurance_limit': 40,
 'loan_requirement': 37,
 'marriage_claim': 26,
 'new_card_reader_cost': 11,
 'new_saving_book': 24,
 'nrfc_account_opening': 18,
 'nrfc_info': 55,
 'precashing_foreign_fixed_deposit': 31,
 'repos_benefits': 53,
 'resident_

In [ ]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [ ]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [ ]:
encoded_output.shape

(1567, 1)

In [ ]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [ ]:
output_one_hot = one_hot(encoded_output)

In [ ]:
output_one_hot.shape

(1567, 56)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)
train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, shuffle = True, test_size = 0.2)

In [ ]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))
print("Shape of test_X = %s and test_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (1002, 22) and train_Y = (1002, 56)
Shape of val_X = (314, 22) and val_Y = (314, 56)
Shape of test_X = (314, 22) and test_Y = (314, 56)


In [ ]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 300, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(64)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(56, activation = "softmax"))
  
  return model

In [ ]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 22, 300)           342300    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_9 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 56)                1848      
Total params: 535,156
Trainable params: 192,856
Non-trainable params: 342,300
_________________________________________________________________


In [ ]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 1002 samples, validate on 314 samples
Epoch 1/100
1002/1002 [==============================] - 6s 6ms/step - loss: 3.9958 - acc: 0.0329 - val_loss: 3.9455 - val_acc: 0.0414

Epoch 00001: val_loss improved from inf to 3.94549, saving model to model.h5
Epoch 2/100
1002/1002 [==============================] - 2s 2ms/step - loss: 3.9077 - acc: 0.0619 - val_loss: 3.8426 - val_acc: 0.0669

Epoch 00002: val_loss improved from 3.94549 to 3.84261, saving model to model.h5
Epoch 3/100
1002/1002 [==============================] - 2s 2ms/step - loss: 3.7821 - acc: 0.0858 - val_loss: 3.6371 - val_acc: 0.1783

Epoch 00003: val_loss improved from 3.84261 to 3.63707, saving model to model.h5
Epoch 4/100
1002/1002 [==============================] - 2s 2ms/step - loss: 3.5774 - acc: 0.1317 - val_loss: 3.4144 - val_acc: 0.1720

Epoch 00004: val_loss improved from 3.63707 to 3.41439, saving model to model.h5
Epoch 5/100
1002/1002 [==============================] - 2s 2ms/step - loss: 3.3267 - acc

In [ ]:
 model = load_model("model.h5")

In [ ]:
def predictions(text):
  
  test_word = word_tokenize(text)
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred

In [ ]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [ ]:
text = "எனது கடன்அட்டையை நான் எவ்வகையில் ரத்து செய்வது?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['எனது', 'கடன்அட்டையை', 'நான்', 'எவ்வகையில்', 'ரத்து', 'செய்வது', '?']
update_personal_details has confidence = 0.7297555
cancel_card has confidence = 0.1370059
card_usage has confidence = 0.04566446
marriage_claim has confidence = 0.035796303
bank_statement_online has confidence = 0.01790954
change_details has confidence = 0.01752386
selan_sure_info has confidence = 0.0040816055
debit_card_requirement has confidence = 0.0027313554
card_foreign_use has confidence = 0.0026993423
SLBFE_info has confidence = 0.0020200538
treasury_bond_important_features has confidence = 0.0017377669
get_monthly_report has confidence = 0.0015380286
suspious_activity has confidence = 0.00058384915
resident_foreign_account_info has confidence = 0.00043325592
saving_atmcard_available has confidence = 0.00012482044
sl_development_bond_benefits has confidence = 6.587701e-05
2in1_acoount_info has confidence = 5.6259312e-05
loan_requirement has confidence = 4.0774892e-05
treasury_bond_information has confidence =

In [ ]:

print(model.test_on_batch(test_X ,test_Y))
model.metrics_names

[0.53003687, 0.9123506]


['loss', 'acc']